### strapi

In [32]:
import requests
import json

response = requests.get("http://13.124.235.216:1337/crawler-keywords",
  headers={
    'Content-Type': 'application/json'
  })

strapi_db = json.loads(response.text)

[Key['Keywords'] for Key in strapi_db]

### es

In [14]:
import sys
import os
sys.path.append(os.path.dirname(os.path.abspath(os.path.dirname('DeepSearch'))))
from connector import es_connector
import json

In [16]:
es = es_connector.ES()

In [39]:
print(es.allIndex())

yellow open klue-mrc-dev                    EKq-LljHQmyceeV4R9LVRw 5 1  5076   0  19.6mb  19.6mb
yellow open news-naver                      cMYtMNzDQOe37VN606U98g 5 1   186   0   2.9mb   2.9mb
yellow open crawling                        3AFnvNLvR2WzhDN8kFW9PQ 5 1    40   0 381.6kb 381.6kb
green  open .kibana_-700776044_deepsearch_1 87d-qHhWRlqV1KivHOsUxg 1 0    23  13  92.5kb  92.5kb
green  open .kibana_1                       lvca5kdxTb2Y1LVm1NCPZg 1 0    66 146 105.7kb 105.7kb
green  open .opendistro_security            VFXXzkbeQGqDMR7T2JDSSg 1 0     9   4  59.5kb  59.5kb
yellow open klue-mrc-train                  IYNT_vsOTNyiHsaJ8thgEg 5 1 12174   0  49.6mb  49.6mb



In [23]:
#es.deleteIndex('news_naver')

In [32]:
# with open('settings.json','r') as f:
#     settings = json.load(f)
    
# with open('mappings.json','r') as f:
#     mappings = json.load(f)
    
# es.createIndex(index='news-naver',settings=settings, mappings=mappings)

In [33]:
from utils import Get_Article_Body, ariticle_reply, get_article_df
import datetime
from selenium.webdriver import Chrome
import json
import sys
import os
sys.path.append(os.path.dirname(os.path.abspath(os.path.dirname('DeepSearch'))))

today = datetime.datetime.now()
yesterday = today + datetime.timedelta(days=-1)


# 검색 키워드 입력
search_keywords = ['삼성생명','라이나생명','ESG','보험','금융']

# 기간 설정
years = [yesterday.strftime("%Y-%m-%d")[:4]]
months = [datetime.datetime.now().strftime("%Y-%m-%d")[5:7]]
#all_days = [yesterday.strftime("%Y-%m-%d")[-2:]]
all_days = list(range(1,19))


driver = Chrome()
article_url_df = get_article_df(years, months, all_days, search_keywords, driver)
#기사 내용 수집
article_dicts = []
reply_dfs = []


In [45]:
article_url_df

,YEAR,MONTH,DAY,url,search_keyword
0,2021,06,1,https://news.naver.com/main/read.nhn?mode=LSD&...,삼성생명
1,2021,06,1,https://news.naver.com/main/read.nhn?mode=LSD&...,삼성생명
2,2021,06,1,https://news.naver.com/main/read.nhn?mode=LSD&...,삼성생명
3,2021,06,1,https://news.naver.com/main/read.nhn?mode=LSD&...,삼성생명
4,2021,06,1,https://news.naver.com/main/read.nhn?mode=LSD&...,삼성생명
...,...,...,...,...,...
188,2021,06,17,https://news.naver.com/main/read.nhn?mode=LSD&...,금융
189,2021,06,17,https://news.naver.com/main/read.nhn?mode=LSD&...,금융
190,2021,06,17,https://news.naver.com/main/read.nhn?mode=LSD&...,금융
191,2021,06,17,https://news.naver.com/main/read.nhn?mode=LSD&...,금융


In [ ]:
for i in range(len(article_url_df)):
    try:
        article_dict = Get_Article_Body(article_url_df['url'][i],driver)
        article_dict['search_keywords'] = article_url_df['search_keyword'][i]
        es.dataInsert(index='news-naver', data=article_dict)
        #reply_df = ariticle_reply()
        #reply_dfs.append(reply_df)
    except:
        continue